In [1]:
!pip install pandas numpy tqdm

In [2]:
import pandas as pd
import numpy as np
import json
from elasticsearch import Elasticsearch
from tqdm import tqdm

In [3]:
ELASTIC_USERNAME = "elastic"
ELASTIC_PASSWORD = "skkudbp"
ELASTIC_PATH = "https://es01:9200"

# You must write your code and result in your **DOCUMENT ALSO**
## Connect to the elasticsearch

In [4]:
######## DO NOT EDIT HERE ########

def connect_to_elastic() -> Elasticsearch:
    client = Elasticsearch(
        ELASTIC_PATH,
        ca_certs = "/home/jovyan/certs/ca/ca.crt",
        basic_auth=(ELASTIC_USERNAME, ELASTIC_PASSWORD),
    )
    return client
client = connect_to_elastic()

######## DO NOT EDIT HERE ########

## **[5pts]** Write mapping for the index

In [12]:
INDEX_NAME = "movie_review"
######## EDIT HERE ########
mapping = {
    "movieId": {
        "type": "integer" # Fill in the blank
    },
    "title": {
        "type": "text"
    },
    "genres": {
        "type": "text"
    },
    "imdbId": {
        "type": "integer"
    },
    "tmdbId": {
        "type": "integer"
    },
    "userId": {
        "type": "integer"
    },
    "rating": {
        "type": "float"
    },
    "timestamp": {
        "type": "date"
    }}
######## EDIT HERE ########

## **[5pts]** Create Index

In [13]:
if not client.indices.exists(index=INDEX_NAME):
    ######## EDIT HERE ########
    client.indices.create(index=INDEX_NAME, body={"mappings":{"properties":mapping}})
    ######## EDIT HERE ########

## **[5pts]** Insert documents

In [14]:
with open('./movie_data.json', 'r') as f:
    movie_json = json.load(f)
    ######## EDIT HERE ########
    for id_doc, document in enumerate(movie_json):
        client.index(index=INDEX_NAME, id=id_doc, body=document)
    ######## EDIT HERE ########

# **[30pts]** Write a query for the given questions and report the results.

## **[10pts]** What is the total <U>number of reviews</U> for the films that contain the <U>both "Action" and "Crime"</U> genres?

In [30]:
query = {
    ######## EDIT HERE ########
    "size": 1000,
    "query": {
        "bool": {
            "filter": [
                {"match": {"genres": "Action"}},
                {"match": {"genres": "Crime"}}
            ]
        }
    }
    ######## EDIT HERE ########
}

######## DO NOT EDIT HERE ########
response = client.search(index=INDEX_NAME, body=query)
print(len(response['hits']['hits']))
######## DO NOT EDIT HERE ########

77


## **[10pts]** What is the total <U>number of reviews</U> for the films that contain the <U>"Sci-Fi"</U> genres and have a <U>rating 4.0 or more but 5.0 or less</U> and report <U>the list of the films</U>.

In [35]:
query = {
    ######## EDIT HERE ########   
    "size": 1000, 
    "query": {
        "bool": {
            "must": [
                {"match": {"genres": "Sci-Fi"}},
                {"range": {"rating": {"gte": 4.0, "lte": 5.0}}}
            ]
        }
    }
    ######## EDIT HERE ########
}

######## DO NOT EDIT HERE ########
response = client.search(index=INDEX_NAME, body=query)
print("Numbers of result:", response['hits']['total']['value'])

movie_set = set()
for res in response['hits']['hits']:
    movie_set.add(res['_source']['title'])
for movie in movie_set:
    print(movie)
######## DO NOT EDIT HERE ########

Numbers of result: 72
Robots (2005)
Truman Show, The (1998)
Doctor Strange (2016)
Unbreakable (2000)
V for Vendetta (2006)
Star Wars: Episode V - The Empire Strikes Back (1980)
Pandorum (2009)
Ghost in the Shell 2.0 (2008)
Aliens (1986)
Island, The (2005)
Interstellar (2014)
2010: The Year We Make Contact (1984)
X-Men (2000)
Blob, The (1958)
Hollow Man (2000)
Piranha (1978)
Stepford Wives, The (1975)
The Hunger Games: Mockingjay - Part 1 (2014)
Star Trek: First Contact (1996)
Altered States (1980)
Avatar (2009)
RoboCop (1987)
Blood of Heroes, The (Salute of the Jugger, The) (1989)
Matrix Revolutions, The (2003)
Final Fantasy: The Spirits Within (2001)
Inception (2010)
Battlestar Galactica (2003)
Flash Gordon (1980)
Bill & Ted's Excellent Adventure (1989)
Cube (1997)
Superman Returns (2006)
Splice (2009)
Dune (1984)
Ex Machina (2015)
The Hunger Games (2012)
Blade Runner (1982)
Back to the Future (1985)
World's End, The (2013)
WALL·E (2008)
Thor: Ragnarok (2017)
Spider-Man (2002)
Demolit

## **[10pts]** Find the <U>10 lowest rated reviews</U> between <U>2005 and 2010</U>, in order.

In [39]:
query = {
    ######## EDIT HERE ########
    "size": 10,
    "query": {
        "bool": {
            "filter": [
                {"range": {"timestamp": {"gte": "2005-01-01", "lte": "2010-12-31"}}}
            ]
        }
    },
    "sort": [
        {"rating": {"order": "asc"}}
    ]
    ######## EDIT HERE ########
}

######## DO NOT EDIT HERE ########
response = client.search(index=INDEX_NAME, body=query)
for hit in response['hits']['hits']:
    print(hit['_source']['title'], hit['_source']['rating'])
######## DO NOT EDIT HERE ########

Raising Cain (1992) 0.5
Titanic (1997) 0.5
Client, The (1994) 0.5
Darkness (2002) 0.5
Sex, Lies, and Videotape (1989) 0.5
Killer Shrews, The (1959) 0.5
Alien: Resurrection (1997) 1.0
Shrek (2001) 1.0
Pee-wee's Big Adventure (1985) 1.0
Legally Blonde (2001) 1.0
